In [ ]:
# this notebook contains the code written by the first beta tester (my mom!) :)

# setting up some boilerplate code to get them started:
# import libraries necessary for building the CNN
import numpy as np
import keras
from keras.layers import Input, Conv2D, Dense, Activation, Flatten
from keras.models import Sequential
from keras.datasets import mnist
# libraries necessary for visualization
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

In [ ]:
# important variables
num_classes = 10 # one for each of the digits from 0-9
batch_size = None # your choice! 
epochs = None # your choice!

In [ ]:
# import mnist data from keras
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# reshape the x data do it's 4 dimensional --> the last dimension represents color channels
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

# we need to normalize the X data before feeding into our model
X_train = X_train/255
X_test = X_test/255

# we also need to convert the Y data into one-hot vectors
Y_train = keras.utils.to_categorical(Y_train, num_classes)
Y_test = keras.utils.to_categorical(Y_test, num_classes)

In [ ]:
# Build the model architecture

In [ ]:
# compile the model

In [ ]:
# train the model

In [ ]:
# plot the model

In [ ]:
# in the next section, we'll take a look at what your model is actually "thinking"

# this is a function that will become helpful in the next cell:

# creating a function to deprocess the image 
from scipy.misc import imsave

# util function to convert a tensor into a valid image
def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to black/white representation
    x *= 255
    #x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [ ]:
# visualize the model's fileters

from keras import backend as K #the backend in this case is tensorflow, since keras is sitting on top of it
                                #so importing the "backend" will allow us to write code in tensorflow that keras
                                #will be able to use for our model

# create a dictionary of the model's layers
layer_dict = dict([(layer.name, layer) for layer in model.layers])
        
#we'll be sending in an image the size of one of the MNIST images
img_width = 28
img_height = 28
        
layer_name = "conv2d_3"
filter_index = 0 #there are 32 filters, so this could be any number in the range 0-31

# create a loss function that we will use to maximize the activation of the specified layer
layer_output = layer_dict[layer_name].output # accesing the output of the specified layer stored in our dictionary
loss = K.mean(layer_output[:,:,:, filter_index]) # averaging all the outputs of the filter --> remember that the filter
                                                # is a 2-D "square" --> the middle two numbers represent height/width
                                                # of that square, the first number represents batch size, and the 
                                                # final number represents the number of filters (we're accessing a
                                                # specific one here)
# compute the gradient of the input picture with respect to this loss.
# this means we'll be updating the pixels of the input image, not the weights of the network --> clever!
grads = K.gradients(loss, model.input)[0] # I don't know what the [0] means at the end of this line
print(grads)

# normalizing the gradient --> we don't want the magnitude of our gradient ascent/descent step to be infuenced heavily 
# by the gradient --> the gradient gives us the direction we want to take --> so normalizing helps the optimization
# algorithm perform better
grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)

iterate = K.function([model.input], [loss, grads]) # this is a fancy way of writing a custom function that takes
                                                # an input image as an input, and returns the loss and gradients
                                                # for that image

# running gradient ascent to maximize the activations of the filter
# generating a grey test image
input_img = np.random.random((1, img_width, img_height, 1)) * 20 + 128.

# Choosing an arbitrary value for "step" --> pretty sure this is like the learning rate
step = 5

# 20 steps of gradient ascent
for i in range(50):
    loss_value, grads_value = iterate([input_img])
    input_img += grads_value * step

In [ ]:
# let's see the filter!
# to see other filters, go and change the filter_index variable above

img = input_img
img = deprocess_image(img)

# the imsave approach
#imsave('%s_filter_%d.png' % (layer_name, filter_index), img)
print(img.shape)


# the matplotlib approach
import matplotlib.pyplot as plt
plt.imshow(img.squeeze())
plt.show()